In [1]:
from flask import Flask, make_response, jsonify
import numpy as np
from flask_cors import CORS
#from flask_compress import Compress
app = Flask(__name__)
#Compress(app)
CORS(app)
from glob import glob
from retrodetect.image_processing import getblockmaxedimage

import argparse
import webbrowser
import os
import re
import retrodetect
import pickle
from datetime import datetime
import numpy as np


In [2]:
pathtoimgsdir = 'Data'
print("Absolute path to images:")
print(os.path.abspath(pathtoimgsdir))
pathtoimgsdir = os.path.abspath(pathtoimgsdir)
pathtoimgs = sorted(glob(pathtoimgsdir+'/*/'))

Absolute path to images:
c:\Users\cs1sch\PycharmProjects\bee_track\bin\Data


In [3]:
pathtoimgsdir

'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data'

In [5]:
if (len(pathtoimgs)==0):
    print("Failed to find any folders in the path, using base path given as camera folder.")
    pathtoimgs = [pathtoimgsdir]
print("Found the following camera folders:")
pathtoimgs
    

Found the following camera folders:


['c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam2\\']

In [6]:
configfilename = pathtoimgsdir+'\config_00_00_00.json'
print(configfilename)

c:\Users\cs1sch\PycharmProjects\bee_track\bin\Data\config_00_00_00.json


#functions for  the cell below
#line 147 - 149 
camera_ids = []
for pti in pathtoimgs:
    camera_ids.append(getorderedcamids(pti))

In [7]:
def getimgfilelist(path,camid=None):
    if camid is not None:
        return sorted(glob('%s/*%s*.np'%(path,camid)))
    else:
        return sorted(glob('%s/*.np'%(path)))

In [ ]:
camera_ids = []
for pti in pathtoimgs:
    camera_ids.append(getorderedcamids(pti))

In [26]:
def getorderedcamids(path):
    """
    Returns camera ids with greyscale ones first
    """
    fns = getimgfilelist(path)
    print(fns)
    cam_ids = list(set([getcamfromfilename(fn) for fn in fns if getcamfromfilename(fn) is not None]))
    return [cam_id for cam_id in cam_ids if guesscamtype(path,cam_id)=='greyscale']+[cam_id for cam_id in cam_ids if guesscamtype(path,cam_id)=='colour']


In [28]:
camera_ids = []
camera_ids.append(getorderedcamids(path))

['c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_06_07.051416.np', 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_06_43.462287__0000.np', 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_06_43.718456__0001.np', 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_06_43.972485__0002.np', 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_07_20.164206__0003.np', 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_07_20.418653__0004.np', 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_47_08.141304__0000.np', 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_47_08.396931__0001.np

EOFError: Ran out of input

In [8]:
path = pathtoimgs[0]
path


'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\'

In [ ]:
fns = getimgfilelist(path)
fns

In [29]:
len(fns)

3452

In [44]:
fns[5]

'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_07_20.418653__0004.np'

In [69]:
for index, fn in enumerate(fns):
    try:
        photo = pickle.load(open(fn,'rb'))
    except EOFError as e:
        continue
    if photo['img'] is None:
        print(index)

0
571
572
573
1360
1677
1678
1679
1680
2386
2426
3409
3410
3451


In [23]:
score = np.abs(np.mean(img[0:-2:2,0:-2:2]/2+img[2::2,2::2]/2-img[1:-2:2,1:-2:2])/np.mean(img))


In [24]:
score

0.0003888568051807193

In [11]:
def getcamfromfilename(fn):
    res = re.findall('photo_object_([0-9A-Z]*)_[0-9]{8}_',fn)
    if len(res)==0:
        return None
    else:
        return res[0]

In [12]:
a = getcamfromfilename(fn)

In [13]:
guesscamtype(path,camid):

'02G14695547'

In [14]:
cam_ids = list(set([getcamfromfilename(fn) for fn in fns if getcamfromfilename(fn) is not None]))
cam_ids


['02G14695547']

In [55]:
def getorderedcamids(path):
    """
    Returns camera ids with greyscale ones first
    """
    fns = getimgfilelist(path)
    cam_ids = list(set([getcamfromfilename(fn) for fn in fns if getcamfromfilename(fn) is not None]))
    return [cam_id for cam_id in cam_ids if guesscamtype(path,cam_id)=='greyscale']+[cam_id for cam_id in cam_ids if guesscamtype(path,cam_id)=='colour']


In [73]:
fn = fns[0]
guesscamtypegetscore(fn)

nan

In [75]:
a = np.NaN
a

nan

In [71]:
np.nanmean([np.NaN,8])

8.0

In [ ]:
score = np.nanmean([guesscamtypegetscore(fn) for fn in getimgfilelist(path,None)[:50:5]])

In [79]:
file_list = getimgfilelist(path,None)[:50:5]
file_list

['c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_06_07.051416.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_07_20.418653__0004.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_47_09.158795__0004.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_48_59.763622__0009.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_49_31.907047__0014.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_50_22.064499__0019.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_50_24.763819__0024.np',
 'c:\\Users\\cs1sch\\PycharmProjects\\bee_track\\bin\\Data\\cam1\\photo_object_02G14695547_20230629_10_50_54.760818_f

In [81]:
score = []
for fn in file_list:
    score.append(guesscamtypegetscore(fn))
score

[nan,
 nan,
 0.0007922565548989565,
 0.001929236505957897,
 0.0013040362867223537,
 0.0004066424389012318,
 0.0007224704892628332,
 0.0006733924904193366,
 0.001319482024889614,
 8.15833063261448e-05]

In [80]:
def guesscamtypegetscore(fn):
    try:
        photo = pickle.load(open(fn,'rb'))
    except EOFError as e:
        return np.NaN


    img = photo['img']
    if img is None:
        score = np.NaN
        #return np.NaN  
    else:
    #e.g. 0.0001 = greyscale, 0.7 = colour
        score = np.abs(np.mean(img[0:-2:2,0:-2:2]/2+img[2::2,2::2]/2-img[1:-2:2,1:-2:2])/np.mean(img))
    return score
    
def guesscamtype(path,camid):
    score = np.nanmean([guesscamtypegetscore(fn) for fn in getimgfilelist(path,camid)[:50:5]])
    if score<0.02:
        return 'greyscale'
    else:
        return 'colour'


In [ ]:
score = np.nanmean([guesscamtypegetscore(fn) for fn in getimgfilelist(path,camid)[:50:5]])


In [74]:
#line 147 - 149 
camera_ids = []
for pti in pathtoimgs:
    camera_ids.append(getorderedcamids(pti))

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'